In [1]:
# import libraries
import yfinance as yf
import pandas as pd

# download historical data for a stock over specific periods
ticker = 'AAPL'
data = yf.download(ticker, start='2020-01-01', end='2023-01-01')

# Convert to single index by using first level of column names only
data.columns = data.columns.get_level_values(0)

# Convert to single index and select/reorder desired columns
data = data[['Open', 'High', 'Low', 'Close', 'Volume']]

# Remove the column multi-index
data.columns = ['Open', 'High', 'Low', 'Close', 'Volume']

# Convert index to date only (removing timestamp)
data.index = data.index.date

# display the first few rows to inspect the datab
print(data.head())

[*********************100%***********************]  1 of 1 completed

                 Open       High        Low      Close     Volume
2020-01-02  74.059998  75.150002  73.797501  75.087502  135480400
2020-01-03  74.287498  75.144997  74.125000  74.357498  146322800
2020-01-06  73.447502  74.989998  73.187500  74.949997  118387200
2020-01-07  74.959999  75.224998  74.370003  74.597504  108872000
2020-01-08  74.290001  76.110001  74.290001  75.797501  132079200


In [2]:
data.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')

In [3]:
# handling missing daa 
data = data.dropna() # drop rows with missing values

# calculate the 14-day SMA and 14-day RSI
data['SMA_14'] = data['Close'].rolling(window=14).mean()

# RSI calculation
delta = data['Close'].diff(1)
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
rs = avg_gain / avg_loss
data['RSI_14'] = 100 - (100 / (1 + rs))

# Normalize 'Close' price for model compatibility (optional)
data['Close_normalized'] = (data['Close'] - data['Close'].mean()) / data['Close'].std()

start_index = 14
data = data[start_index:]
# Display the data with new indicators
print(data[['Close', 'SMA_14', 'RSI_14', 'Close_normalized']].head(20))

                Close     SMA_14     RSI_14  Close_normalized
2020-01-23  79.807503  77.628929  71.902570         -1.652277
2020-01-24  79.577499  78.001786  75.401481         -1.659801
2020-01-27  77.237503  78.165179  59.513444         -1.736356
2020-01-28  79.422501  78.509822  67.412481         -1.664872
2020-01-29  81.084999  78.887500  68.465163         -1.610483
2020-01-30  80.967499  79.141786  63.879136         -1.614327
2020-01-31  77.377502  79.127143  49.368860         -1.731776
2020-02-03  77.165001  78.978929  42.987505         -1.738728
2020-02-04  79.712502  79.089108  54.739602         -1.655385
2020-02-05  80.362503  79.269644  57.618700         -1.634120
2020-02-06  81.302498  79.447680  57.529076         -1.603367
2020-02-07  80.007500  79.470894  50.957282         -1.645734
2020-02-10  80.387497  79.559822  53.702040         -1.633302
2020-02-11  79.902496  79.593929  51.402949         -1.649169
2020-02-12  81.800003  79.736250  55.375694         -1.587091
2020-02-

In [4]:
# Step 1: Initialize variables for backtesting
initial_balance = 10000  # Starting with ₹10,000
balance = initial_balance
position = 0  # Start with no stock position
returns = []  # To store returns from each trade

# Step 2: Iterate through data to simulate the trading strategy
for i in range(1, len(data)):
    if data['RSI_14'].iloc[i] < 30 and position == 0:  # Buy condition
        # Buy stock
        position = balance / data['Close'].iloc[i]
        balance = 0  # Invest all balance into stock

    elif data['RSI_14'].iloc[i] > 70 and position > 0:  # Sell condition
        # Sell stock
        balance = position * data['Close'].iloc[i]
        position = 0  # No stock position after selling
        returns.append(balance - initial_balance)  # Calculate profit or loss

# Step 3: Calculate overall performance
final_balance = balance if position == 0 else position * data['Close'].iloc[-1]
total_return = final_balance - initial_balance

# Display results
print(f"Initial Balance: ₹{initial_balance}")
print(f"Final Balance: ₹{final_balance}")
print(f"Total Return: ₹{total_return}")
print(f"Return Percentage: {((final_balance - initial_balance) / initial_balance) * 100:.2f}%")


Initial Balance: ₹10000
Final Balance: ₹8628.465887530288
Total Return: ₹-1371.5341124697115
Return Percentage: -13.72%
